In [2]:
import geopandas as gpd
from pyproj import CRS
import matplotlib.pyplot as plt

from pathlib import Path

In [3]:
dir_data = Path("..")/ "data"/ "danum" 
fp = dir_data/ "crowns_vector" / "dalponte_2013.json"

## What's the problem here? How do you fix it?

In [4]:
data = gpd.read_file(fp)

In [9]:
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
data.head()

,DN,geometry
0,22,"POLYGON ((588123.000 548713.500, 588123.000 54..."
1,18,"POLYGON ((588548.000 548713.500, 588548.000 54..."
2,5,"POLYGON ((587595.000 548713.500, 587595.000 54..."
3,19,"POLYGON ((587205.500 548713.500, 587205.500 54..."
4,13,"POLYGON ((588284.500 548713.500, 588284.500 54..."


Hint: this is the bounding box

In [21]:
fp_bounds = dir_data/"danum_loc" / "Danum_50ha_0_0.shp"
data_bounds = gpd.read_file(fp_bounds)
bbox = data_bounds.geometry[0]

Hint 2: [UTM](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system)

Hint 3: [link to specific part of pyproj documentation](https://pyproj4.github.io/pyproj/stable/examples.html#find-utm-crs-by-latitude-and-longitude)

In [32]:
# Looking up the CRS based on lat lon
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info

print(bbox.coords[0])
lon = bbox.coords[0][0]
lat = bbox.coords[0][1]


utm_crs_list = query_utm_crs_info(
    datum_name="WGS 84",
    area_of_interest=AreaOfInterest(
        west_lon_degree=lon,
        south_lat_degree=lat,
        east_lon_degree=lon,
        north_lat_degree=lat,
    ),
)
new_crs = CRS.from_epsg(utm_crs_list[0].code)


(117.79208052796504, 4.959671580931951)


<Projected CRS: EPSG:32650>
Name: WGS 84 / UTM zone 50N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 114°E and 120°E, northern hemisphere between equator and 84°N, onshore and offshore. Brunei. China. Hong Kong. Indonesia. Malaysia - East Malaysia - Sarawak. Mongolia. Philippines. Russian Federation. Taiwan.
- bounds: (114.0, 0.0, 120.0, 84.0)
Coordinate Operation:
- name: UTM zone 50N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [33]:
## Or finding the code manually
new_crs = CRS("EPSG:32650")

In [34]:
## Correcting the crs in place
data_wrong = data.copy()
data.crs = new_crs
data.crs

<Projected CRS: EPSG:32650>
Name: WGS 84 / UTM zone 50N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 114°E and 120°E, northern hemisphere between equator and 84°N, onshore and offshore. Brunei. China. Hong Kong. Indonesia. Malaysia - East Malaysia - Sarawak. Mongolia. Philippines. Russian Federation. Taiwan.
- bounds: (114.0, 0.0, 120.0, 84.0)
Coordinate Operation:
- name: UTM zone 50N
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## Play around with different world projections

[Top 10 World Map Projections – The Future Mapping Company](https://futuremaps.com/blogs/news/top-10-world-map-projections)

#### Reference

- http://geopandas.org/projections.html
- https://pyproj4.github.io/pyproj/dev/api/crs.html
- https://spatialreference.org/
- https://epsg.io/

In [ ]:
# Read in data
fp = gpd.datasets.get_path("naturalearth_lowres")
world = gpd.read_file(fp)

In [ ]:
# #Template
# proj = ...

# # Re-project and plot
# world.to_crs(proj).plot()

# # Remove x and y axis
# plt.axis('off')
# plt.title("Proj")